# DCA in backtest
Here to more simulate the trading, we use backtest module and implement a DCA strategy to verify it.
The goodness of `class Market` is we only need to care about how to react with it.  This allow user simulate more complicate trading operation like
sending order, cancel order,...etc

In [1]:
# Run it when crypto-backtest is updated, and then restart the kernel
!pip uninstall crypto-backtest -y
!pip install git+https://github.com/BullionBear/crypto-backtest.git

Found existing installation: crypto-backtest 0.0.10
Uninstalling crypto-backtest-0.0.10:
  Successfully uninstalled crypto-backtest-0.0.10
  Cloning https://github.com/BullionBear/crypto-backtest.git to /tmp/pip-req-build-m_mtt_q7
  Running command git clone --filter=blob:none --quiet https://github.com/BullionBear/crypto-backtest.git /tmp/pip-req-build-m_mtt_q7
  Resolved https://github.com/BullionBear/crypto-backtest.git to commit 9efc5b6446d7b1e26cab616bafd55efe658b417b
  Preparing metadata (setup.py) ... done
  Created wheel for crypto-backtest: filename=crypto_backtest-0.0.12-py3-none-any.whl size=10035 sha256=ae1189bb672f5b6b037628e718fd7ed0cc6245e8624e87270c04d82f252726f2
  Stored in directory: /tmp/pip-ephem-wheel-cache-734b1ecf/wheels/14/d1/5f/193347d44b51670dd287000ac535cdb7a8fc3f84e426532382
Successfully built crypto-backtest

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [1]:
import backtest
backtest.__version__

'develop_0.0.10+'

In [15]:
from backtest.components.core import ZipKLineIterator
from backtest.components import KLineMarket

from datetime import datetime, timezone

start_time = datetime(2023, 3, 8, tzinfo=timezone.utc).timestamp() * 1000
end_time = datetime(2023, 4, 8, tzinfo=timezone.utc).timestamp() * 1000
data_source = '/home/yite/crypto_data/binance/data'
kline_iterator = ZipKLineIterator('BTCUSDT', start_time, end_time, data_source)
market = KLineMarket(kline_iterator)

In [16]:
market.get_ts()

1678237199999

In [17]:
from backtest.models import MarketOrder

total_usdt = 1_000_000
invest_usdt = 30_000

while True:
    klines = []
    while market.is_kline():
        klines.append(market.get_kline())
    for kline in klines:
        dt = datetime.fromtimestamp(kline.close_time / 1000)
        price = kline.close
        amount = price / invest_usdt
        if dt.hour == 0:
            market_order = MarketOrder(
                timestamp=kline.close_time, 
                base='BTC', 
                quote='USDT', 
                side='BUY', 
                amount=amount
            )
            market.send_market(market_order)
    if not market.next():
        break
    

In [18]:
trades = []
while market.is_trade():
    trade = market.get_trade()
    trades.append(trade)

In [26]:
import pandas as pd
pd.DataFrame(list(map(dict, trades)))

,timestamp,execution_id,base,quote,side,filled,order_id
0,1678298399999,882bef62-a626-4c6b-af76-0f22a35db411,BTC,USDT,BUY,0.738020,a2d4388c-cc56-4d96-860e-5eab093083a5
1,1678384799999,5d492093-35b2-4069-9964-94b695e36d14,BTC,USDT,BUY,0.715962,7200e561-7fe7-4017-a60c-b003cc1932ab
2,1678471199999,aa517bba-8347-43ed-8e9e-ccd9e239a059,BTC,USDT,BUY,0.667212,85b183c7-07ad-4e17-92ed-6371d59c7b12
3,1678557599999,2f62c83b-b627-4339-9fdd-9b9f083581ba,BTC,USDT,BUY,0.669704,98999d67-a441-4a87-8db0-d866fb8293fc
4,1678643999999,6c08a47d-1edf-4767-824e-2ff711613406,BTC,USDT,BUY,0.679652,26330fdc-d357-40c9-a127-e6c62238266d
5,1678730399999,4f946920-ce55-45a0-a17b-aa250f7708a6,BTC,USDT,BUY,0.798505,3db17b92-d933-4c7e-a35a-6f07c12b6de5
6,1678816799999,1b61f63c-5fcc-4e37-8d70-a1ff6a737656,BTC,USDT,BUY,0.862240,43b54acc-9d89-4f88-b806-2e20c775bc2c
7,1678903199999,98270af4-ca38-4bf9-bf15-f31bac6acab6,BTC,USDT,BUY,0.803490,b657f6b1-d0a3-4a28-87b7-85262c31ada3
8,1678989599999,fe899932-97d0-4edd-af75-47e8ce75f4c1,BTC,USDT,BUY,0.826026,5c06a130-46d9-45af-8867-1013901f3b3c
9,1679075999999,35a5b9f9-8418-4d95-aa4e-ded2305e1ac9,BTC,USDT,BUY,0.879799,2c65b798-0337-4288-8b59-cf3e5255917b


In [8]:
market_order = MarketOrder(
    timestamp=1640995200,  # Example timestamp
    base="BTC",
    quote="USD",
    side="buy",
    amount=1.0
    # order_id is optional since it defaults to uuid4
)